In [43]:
import pandas as pd
import numpy as np

In [5]:
train = pd.read_csv('input/train.csv')
test = pd.read_csv('input/test.csv')

In [6]:
train.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
X_train = train.drop(['label'], axis=1)
y_train = train['label']

In [11]:
X_train.shape

(42000, 784)

In [12]:
y_train.shape

(42000,)

In [15]:
# 欠損値確認
X_train.isna().any().describe()

count       784
unique        1
top       False
freq        784
dtype: object

In [17]:
test.isna().any().describe()

count       784
unique        1
top       False
freq        784
dtype: object

In [19]:
X_train /= 255.0
test /= 255.0

In [24]:
X_train = X_train.values.reshape(-1, 28, 28, 1)
test = test.values.reshape(-1, 28, 28, 1)

In [25]:
X_train.shape

(42000, 28, 28, 1)

In [31]:
from keras.utils import to_categorical
y_train = to_categorical(y_train, num_classes=10)

In [32]:
from sklearn.model_selection import train_test_split

X_train, X_validate, y_train, y_validate = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

In [33]:
X_train.shape

(26880, 28, 28, 1)

In [34]:
y_train.shape

(26880, 10)

In [35]:
from tensorflow import keras
from tensorflow.keras import layers

inputs = keras.Input(shape=(28, 28, 1))
x = layers.Conv2D(filters=32, kernel_size=3, activation="relu")(inputs)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=64, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=128, kernel_size=3, activation="relu")(x)
x = layers.Flatten()(x)
outputs = layers.Dense(10, activation="softmax")(x)
model = keras.Model(inputs=inputs, outputs=outputs)

In [36]:
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 28, 28, 1)]       0         
                                                                 
 conv2d_5 (Conv2D)           (None, 26, 26, 32)        320       
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 13, 13, 32)       0         
 2D)                                                             
                                                                 
 conv2d_6 (Conv2D)           (None, 11, 11, 64)        18496     
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 5, 5, 64)         0         
 2D)                                                             
                                                                 
 conv2d_7 (Conv2D)           (None, 3, 3, 128)         7385

In [37]:
model.compile(optimizer="Adam", loss="categorical_crossentropy", metrics=["accuracy"])

In [40]:
epochs = 3
batch_size = 64

In [41]:
history = model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(X_validate, y_validate))

Epoch 1/3
420/420 [==============================] - 5s 12ms/step - loss: 0.3551 - accuracy: 0.8923 - val_loss: 0.3332 - val_accuracy: 0.8911
Epoch 2/3
420/420 [==============================] - 5s 12ms/step - loss: 0.2842 - accuracy: 0.9120 - val_loss: 0.2584 - val_accuracy: 0.9182
Epoch 3/3
420/420 [==============================] - 5s 12ms/step - loss: 0.2243 - accuracy: 0.9299 - val_loss: 0.1989 - val_accuracy: 0.9324


In [44]:
results = model.predict(test)
# returns = np.argmax(results, axis=1)
# results = pd.Series(results)

875/875 [==============================] - 2s 2ms/step


ValueError: Data must be 1-dimensional